In [60]:
insert_batch_size = 1000
query_count = 20


dataverse = "LocationDb"
table_name = "Locations_RTree_Constant"

dbHost = "http://localhost:19002"
benchmark_plan_path = f"commands/benchmark_plan_{dataverse}_{table_name}_{insert_batch_size}_{query_count}.sql"

# None means no limit
debug_max_batches = 10

In [61]:
data =[]
with open("data/point_data_centroid.csv", "r") as f:
    f.readline()
    for line in f:
        row = line.strip().split(",")
        id = row[0]
        locationX = row[1]
        locationY = row[2]
        description = row[3]
        data.append([id, locationX, locationY, description])

In [62]:
import random
import requests

def point(x, y):
    return f'point("{x},{y}")'

def rectangle(x1, y1, x2, y2):
    return f'rectangle("{x1},{y1} {x2},{y2}")'

def generate_insert_command(batch):
    query_lines = []
    query_lines.append(f"INSERT INTO {table_name} ([")
    for row in batch:
        id = row[0]
        location_x = row[1]
        location_y = row[2]
        description = row[3]
        is_last = (row == batch[-1])
        query_lines.append(f'{{"id": {id}, "location": {point(location_x, location_y)}, "description": "{description}"}}{ "" if is_last else "," }')
    query_lines.append("])")
    
    return "".join(query_lines)

def generate_query_commands(inserted_points):
    queries = []
    selected_points = random.sample(inserted_points, query_count)
    for selected_point in selected_points:
        location_x = selected_point[0]
        location_y = selected_point[1]
        offset = random.randint(0, 100)
        start_x = float(location_x) - offset
        start_y = float(location_y) - offset
        end_x = float(location_x) + offset
        end_y = float(location_y) + offset
        
        query = f"SELECT location, description FROM {table_name} WHERE spatial_intersect(location, {rectangle(start_x, start_y, end_x, end_y)});";
        queries.append(query)
    return queries

In [63]:
inserted_points = []
current_batch = 0
commands = []
while True:
    if debug_max_batches is not None and current_batch >= debug_max_batches:
        break
    batch_to_insert = data[current_batch * insert_batch_size:(current_batch + 1) * insert_batch_size]
    if not batch_to_insert: # no more data to insert
        break
    
    insert_command = generate_insert_command(batch_to_insert)
    inserted_points.extend(map(lambda x: (x[1], x[2]), batch_to_insert))
    
    query_commands = generate_query_commands(inserted_points)
    commands.append(insert_command)
    commands.extend(query_commands)
    
    current_batch += 1
    
# write the commands to a file
with open(benchmark_plan_path, "w") as f:
    for command in commands:
        f.write(command + "\n")
    

In [ ]:
def execute_command(command):
    return requests.post(f"{dbHost}/query/service", data={
        'statement': command,
        'dataverse': dataverse,
    })
    
with open(benchmark_plan_path, "r") as f:
    for line in f:
        command = line.strip()
        if command:
            print(f"Executing: {command}")
            response = execute_command(command)
            if response.status_code != 200:
                print(f"Error executing command: {response.text}")
            else:
                print("Command executed successfully")
                print(response.json())
            print()

Executing: INSERT INTO Locations_RTree_Constant ([{"id": 1, "location": point("71.60207635350525,54.78643014239917"), "description": "8635a3cf61"},{"id": 2, "location": point("68.07547055464238,40.23959944486414"), "description": "a38db8bb2a"},{"id": 3, "location": point("22.71972581720911,48.451325419849496"), "description": "09ee56c7cb"},{"id": 4, "location": point("29.491294654319063,46.07657549119183"), "description": "d712c4bd92"},{"id": 5, "location": point("79.38921734504402,52.74566659600047"), "description": "6e0c10f6e4"},{"id": 6, "location": point("6.359217615158059,43.142364551113644"), "description": "c1bc6b682a"},{"id": 7, "location": point("1.9459106666618595,49.15726517327247"), "description": "9260faec9d"},{"id": 8, "location": point("7.758160430065118,43.808794503076086"), "description": "9d017e98ff"},{"id": 9, "location": point("-0.9758063970803335,51.474531240130844"), "description": "8acf845e2f"},{"id": 10, "location": point("66.95163331926233,39.698323306344484"),